# Basic Example: Prompt+Model+OutputParser

- Author: [ChangJun Lee](https://www.linkedin.com/in/cjleeno1/)
- Design: []()
- Peer Review: [Erika](https://github.com/ErikaPark)
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-4/sub-graph.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239937-lesson-2-sub-graphs)


## Overview

The most fundamental and commonly used case involves linking a prompt template with a model. To illustrate how this works, let us create a chain that asks for the capital cities of various countries.


### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Utilizing Prompt Templates](#utilizing-prompt-templates)
- [Chain Creation](#chain-creation)

### References

- [LangChain ChatOpenAI API reference](https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html)
- [LangChain Core Output Parsers](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.list.CommaSeparatedListOutputParser.html#)
- [Python List Tutorial](https://docs.python.org/3.13/tutorial/datastructures.html)
---

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [7]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai",
        "langchain_community",
    ],
    verbose=False,
    upgrade=False,
)

In [6]:
# Set local environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "",
    }
)

Environment variables have been set successfully.


---
You can alternatively set `OPENAI_API_KEY` in `.env` file and load it. 

[Note] This is not necessary if you've already set `OPENAI_API_KEY` in previous steps.

In [2]:
# Configuration File for Managing API Key as an Global Environment Variable
from dotenv import load_dotenv

# Load API KEY Information
load_dotenv(override=True)

True

In [7]:
# Set up LangSmith tracking: https://smith.langchain.com
from langsmith import utils

utils.tracing_is_enabled()

False

## Utilizing Prompt Templates

`PromptTemplate`

- A prompt template is used to create a complete prompt string by incorporating the user's input variables.
- Usage
  - `template`: A template string is a predefined format where curly braces '{}' are used to represent variables.

  - `input_variables`: The names of the variables to be inserted within the curly braces are defined as a list.

`input_variables`

- `input_variables` is a list that defines the names of the variables used in the `PromptTemplate`.

In [8]:
from langchain_core.prompts import PromptTemplate

The `from_template()` method is used to create a `PromptTemplate` object.

In [9]:
# Define template
template = "What is the capital of {country}?"

# Create a `PromptTemplate` object using the `from_template` method.
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}?')

In [10]:
# Generate the prompt.
prompt = prompt_template.format(country="Korea")
prompt

'What is the capital of Korea?'

In [11]:
# Generate the prompt.
prompt = prompt_template.format(country="USA")
prompt

'What is the capital of USA?'

In [12]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

## Chain Creation

### LCEL (LangChain Expression Language)

Here, we use LCEL to combine various components into a single chain.

![lcel.png](./assets/lcel.png)

```
chain = prompt | model | output_parser
```

The `|` symbol works similarly to the [Unix pipe operator](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>), linking different components and passing the output of one component as the input to the next.

In this chain, user input is passed to the prompt template, and the output from the prompt template is then forwarded to the model. By examining each component individually, you can understand what happens at each step.

In [14]:
# Create the prompt as a `PromptTemplate` object.
prompt = PromptTemplate.from_template("Please explain {topic} in simple terms.")


# Combine the prompt and model into a chain
chain = prompt | model

### Calling `invoke()`

- Input values are provided in the form of a Python dictionary (key-value pairs).  
- When calling the `invoke()` function, these input values are passed as arguments.

In [15]:
# Set the topic in the `input` dictionary to 'The Principles of Learning in Artificial Intelligence Models'.
input = {"topic": "The Principles of Learning in Artificial Intelligence Models"}

In [16]:
# Connect the `prompt` object and the `model` object using the pipe (`|`) operator.  
# Use the `invoke` method to pass the `input`.  
# This will return the message generated by the AI model.
chain.invoke(input)

AIMessage(content="Sure! The Principles of Learning in Artificial Intelligence (AI) Models can be understood as the basic ideas that guide how these models learn from data. Here are some key principles explained in simple terms:\n\n1. **Data-Driven Learning**: AI models learn from data. The more relevant and high-quality data they have, the better they can learn patterns and make predictions. Think of it like a student learning from textbooks; the more information they have, the more they can understand.\n\n2. **Generalization**: Once an AI model learns from specific examples, it should be able to apply that knowledge to new, unseen examples. This is like a student who learns math concepts and can solve new problems that they haven't practiced before.\n\n3. **Feedback and Adjustment**: AI models often use feedback to improve their performance. When they make mistakes, they can adjust their understanding based on the feedback they receive, similar to how a student learns from their erro

Below is an example of outputting a streaming response:

In [17]:
# Request for Streaming Output
answer = chain.stream(input)

# Streaming Output
for token in answer:
    print(token.content, end="", flush=True)

Sure! The Principles of Learning in Artificial Intelligence (AI) Models can be understood as the basic ideas that guide how AI systems learn from data and improve their performance over time. Here are some key principles explained in simple terms:

1. **Data is Key**: AI models learn from data. The more quality data they have, the better they can learn. Think of it like a student studying for a test; the more information they have, the better they can prepare.

2. **Learning from Examples**: AI models often learn by looking at examples. For instance, if you show an AI many pictures of cats and dogs, it can learn to tell the difference between them. This is similar to how we learn by seeing different examples in school.

3. **Feedback Loop**: AI models improve through feedback. When they make a mistake, they can learn from it. For example, if an AI incorrectly identifies a cat as a dog, it can adjust its understanding based on that mistake, much like how we learn from our errors.

4. **

### Output Parser

An **Output Parser** is a tool designed to transform or process the responses from an AI model into a specific format. Since the model's output is typically provided as free-form text, an **Output Parser** is essential to convert it into a structured format or extract the required data.


In [18]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser() # Directly returns the model's response as a string without modification.

An output parser is added to the chain.

In [19]:
# A processing chain is constructed by connecting the prompt, model, and output parser.
chain = prompt | model | output_parser

In [20]:
# Use the invoke method of the chain object to pass the input
chain.invoke(input)

"Sure! The principles of learning in artificial intelligence (AI) models can be understood as the basic ideas that guide how these models learn from data. Here are some key principles explained in simple terms:\n\n1. **Data is Key**: AI models learn from data. The more relevant and high-quality data you provide, the better the model can learn and make predictions.\n\n2. **Learning from Examples**: AI models often learn by looking at examples. For instance, if you want a model to recognize cats in pictures, you show it many pictures of cats and non-cats. The model learns to identify patterns that distinguish cats from other objects.\n\n3. **Feedback Loop**: Many AI models improve through feedback. After making predictions, they compare their results to the correct answers. If they get it wrong, they adjust their understanding to do better next time.\n\n4. **Generalization**: A good AI model doesn’t just memorize the examples it sees; it learns to generalize from them. This means it can 

In [21]:
# Request for Streaming Output
answer = chain.stream(input)

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

Sure! The Principles of Learning in Artificial Intelligence (AI) Models can be understood as the basic ideas that guide how AI systems learn from data and improve their performance over time. Here are some key principles explained in simple terms:

1. **Data is Key**: AI models learn from data. The more quality data they have, the better they can learn. Think of it like a student studying for a test; the more information they have, the better they can prepare.

2. **Learning from Examples**: AI models learn by looking at examples. For instance, if you want an AI to recognize cats in pictures, you show it many pictures of cats and non-cats. Over time, it learns to identify the features that make a cat a cat.

3. **Feedback Loop**: AI models often use feedback to improve. After making a prediction (like guessing if a picture has a cat), they check if they were right or wrong. This feedback helps them adjust and make better guesses in the future.

4. **Generalization**: A good AI model do

### Applying and Modifying Templates

- The prompt content below can be **modified** as needed for testing purposes.  
- The `model_name` can also be adjusted for testing.

In [22]:
template = """
You are a seasoned English teacher with 10 years of experience. Please write an English conversation suitable for the given situation.  
Refer to the [FORMAT] for the structure.

#SITUATION:
{question}

#FORMAT:
- Dialogue in English:
- Explanation of the Dialogue: 
"""

# Generate the prompt using the PromptTemplate
prompt = PromptTemplate.from_template(template)

# Initialize the ChatOpenAI model.
model = ChatOpenAI(model_name="gpt-4o-mini")

# Initialize the string output parser.
output_parser = StrOutputParser()

In [23]:
# Construct the chain.
chain = prompt | model | output_parser

In [24]:
# Execute the completed Chain to obtain a response.
print(chain.invoke({"question": "I want to go to a restaurant and order food."}))

- **Dialogue in English:**

**Waiter:** Good evening! Welcome to Bella Italia. How many are in your party?  
**You:** Just one, please.  
**Waiter:** Right this way. Here’s your menu. Can I get you something to drink while you look it over?  
**You:** Yes, I’d like a glass of water, please.  
**Waiter:** Sure! I’ll be right back with that.  
**You:** Thank you!  
**(After a few minutes)**  
**Waiter:** Are you ready to order?  
**You:** Yes, I’d like the spaghetti carbonara, please.  
**Waiter:** Great choice! Would you like any appetizers or dessert with that?  
**You:** I’ll have a side salad as an appetizer, but no dessert for me today.  
**Waiter:** Perfect! I’ll get that order in for you.  
**You:** Thank you!  
**Waiter:** You’re welcome. Enjoy your meal!

- **Explanation of the Dialogue:**
In this conversation, the exchange takes place in a restaurant setting where the customer is being greeted by the waiter. The dialogue begins with a warm welcome and inquiry about the number o

In [25]:
# Execute the completed Chain to obtain a response  
# Request for Streaming Output
answer = chain.stream({"question": "I want to go to a restaurant and order food."})

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

- Dialogue in English:
**Customer:** Hi there! Can I see the menu, please?  
**Waiter:** Of course! Here you go. Would you like any recommendations?  
**Customer:** Yes, please. What do you recommend for a main dish?  
**Waiter:** Our grilled salmon is very popular, and it's served with seasonal vegetables.  
**Customer:** That sounds delicious! I’ll have the grilled salmon. Can I also get a side salad?  
**Waiter:** Absolutely! What dressing would you like for your salad?  
**Customer:** I'll have the balsamic vinaigrette, please.  
**Waiter:** Great choice! Would you like anything to drink?  
**Customer:** Yes, a glass of iced tea, please.  
**Waiter:** Perfect! I’ll place your order now. It will be ready shortly.

- Explanation of the Dialogue:
In this conversation, the customer initiates the interaction by asking for the menu, indicating they are ready to order. The waiter responds promptly and offers assistance, showcasing good customer service. The customer seeks recommendations,

In [26]:
# This time, set the question to 'Ordering Pizza in the US' and execute it.  
# Request for Streaming Output
answer = chain.stream({"question": "Ordering Pizza in the US"})

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

- Dialogue in English:
**Customer:** Hi there! I’d like to place an order for a pizza, please.

**Pizza Shop Employee:** Sure! What size would you like? We have small, medium, large, and extra-large.

**Customer:** I’ll go with a large, please. 

**Pizza Shop Employee:** Great choice! What toppings would you like on your pizza?

**Customer:** Can I have pepperoni, mushrooms, and black olives?

**Pizza Shop Employee:** Absolutely! Would you like any extra cheese or a special crust?

**Customer:** Yes, I’d love extra cheese, and can I get a stuffed crust?

**Pizza Shop Employee:** You got it! So that’s a large pizza with pepperoni, mushrooms, black olives, extra cheese, and stuffed crust. Would you like anything to drink?

**Customer:** Yes, can I get a two-liter bottle of soda?

**Pizza Shop Employee:** Sure! What kind of soda would you like?

**Customer:** I'll take a cola, please.

**Pizza Shop Employee:** Perfect! Your total comes to $25. Would you like to pick it up or have it deliv